
## Creating Databricks Secrets

### 1. Using the CLI

● **Create a secret scope**  
```bash
databricks secrets create-scope <scope-name>
```
● **Add a secret to the scope**
```bash
databricks secrets put-secret <scope-name> <key-name>
```

### 2. Using the SDK

● Import the WorkspaceClient
```python
from databricks.sdk import WorkspaceClient
workspace_client: WorkspaceClient = WorkspaceClient()
```
● Create the scope
```python
 workspace_client.secrets.create_scope(scope="my-scope")
```
●  Add a secret
```python
from databricks.sdk.service.workspace import PutSecretRequest
workspace_client.secrets.put_secret(
  request=PutSecretRequest(
    scope="my-scope",
    key="api-key",
    string_value="your-secret-value",
  ),
)
```

> **Tip:** Never hardcode secrets. Use scopes and keys to manage sensitive values securely.

In [0]:
from databricks.sdk import WorkspaceClient
from typing import Final, List

workspace_client: WorkspaceClient = WorkspaceClient()

# First we create the scope if it does not already exists
scope_name: Final[str] = "my-scope"
existing_scopes: List[str] = [scope.name for scope in dbutils.secrets.listScopes()]
if scope_name not in existing_scopes:    
    print(f'Creating scope "{scope_name}"')
    workspace_client.secrets.create_scope(scope=scope_name)
    print(f'✅ Scope created')
else:
    print(f'✅ Scope "{scope_name}" already exists')

# Then we add a secret to the scope
secret_key: Final[str] = "my-secret"
secret_value: Final[str] = "my-secret-value2" # ← Never hardcode secrets like this
existing_keys: List[str] = [scope.key for scope in dbutils.secrets.list(scope=scope_name)]
if secret_key in existing_keys:
    print(f'Secret "{secret_key}" already exists in scope "{scope_name}", updating value...')
    workspace_client.secrets.put_secret(scope=scope_name, key=secret_key, string_value="my-secret-value")
    print(f'✅ Secret updated')
else:
    print(f'Adding secret "{secret_key}" to scope "{scope_name}"')
    workspace_client.secrets.put_secret(scope=scope_name, key=secret_key, string_value="my-secret-value")
    print(f'✅ Secret added')

## Accessing Secret Values in Databricks

Use `dbutils.secrets.get` to securely fetch secret values:
```python
api_key: str = dbutils.secrets.get(scope="my-scope", key="api-key")
```

> Secret values are hidden from notebook output, unless they are altered.

In [0]:
secret_value: str = dbutils.secrets.get(scope="my-scope", key="test-key")
print(f"Secret value (redacted): {secret_value}  Secret values are exposed as [REDACTED]")
print(f"Secret value uppercased: {secret_value.upper()}  Any value no longer equal to original will be exposed")
print(f"Secret value lowercased: {secret_value.lower()}  Equals original secret value, thus redacted")
derived_value: str = "this " + secret_value + " has been modified"
print(f"Derived value: {derived_value}  Values are still hidden when used as a part of larger entities")
altered_value: str = secret_value[0:16] + "\0" + secret_value[16:]
print(f"Altered value: {altered_value}  Value has been altered, thus exposed")
hidden_value: str = altered_value.replace("\0", "")
print(f"Restored value: {hidden_value}  Restoring to the original value hides it again, even when exposed before")

This cell demonstrates how it is possible to ingest all secret scopes and their permissions for use in a security dashboard, for example.

In [0]:
from pyspark.sql.types import StructType, StructField, StringType

scopes: List[str] = [s.name for s in workspace_client.secrets.list_scopes()]

rows: list[tuple[str, str, str]] = [
    (scope.name, acl.principal, str(acl.permission))
    for scope in workspace_client.secrets.list_scopes()
    for acl in workspace_client.secrets.list_acls(scope.name)
]

schema: StructType = StructType([
    StructField("scope", StringType(), True),
    StructField("principal", StringType(), True),
    StructField("permission", StringType(), True),
])

df = spark.createDataFrame(rows, schema)
display(df)